# Is the FIFA World Cup Draw Truly Random?

### Pratik Kulkarni - 19400570
### Shrey Parekh - 18706941
\
***Project for 301111 Discovery Project***\
\
**Supervisor: Dr. Russell Thomson**\
\
***School of Computer, Data and Mathematical Sciences,***\
***Western Sydney University***

**Spring, 2021**
***

In [1]:
#Here we import the libraries we will be using for this project.

import pandas as pd
import numpy as np
import math
import seaborn as sns
import pandas as pd

from scipy import stats
from matplotlib import pyplot as plt

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

***

## 1 Fifa Rankings

In [2]:
#Here we load the data set and store in the variable fifaRanks
#We use the pandas library to read the data set
fifaRanks = pd.read_csv("fifa_ranking-2020-12-10.csv")

In [3]:
display(fifaRanks)

,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,43885,104,Swaziland,SWZ,10,0,0,CAF,1992-12-31
1,43972,42,Turkey,TUR,31,0,0,UEFA,1992-12-31
2,43952,43,Northern Ireland,NIR,31,0,0,UEFA,1992-12-31
3,43945,44,Finland,FIN,31,0,0,UEFA,1992-12-31
4,43976,45,Australia,AUS,29,0,0,AFC,1992-12-31
...,...,...,...,...,...,...,...,...,...
62419,43840,74,United Arab Emirates,UAE,1326,1326,0,AFC,2020-12-10
62420,43812,75,China PR,CHN,1323,1323,0,AFC,2020-12-10
62421,1930407,76,Curaçao,CUW,1313,1313,0,CONCACAF,2020-12-10
62422,43901,51,Costa Rica,CRC,1427,1427,0,CONCACAF,2020-12-10


In [4]:
# convert date column into date format
fifaRanks['rank_date'] = pd.to_datetime(fifaRanks['rank_date'])

# Change old team name of Côte d'Ivoire to Ivory Coast
fifaRanks[['country_full']] = fifaRanks[['country_full']].replace("Côte d'Ivoire", "Ivory Coast")

# filter rows on the basis of date
fifaRanks_filtered = (fifaRanks['rank_date'] > '1994-01-01') & (fifaRanks['rank_date'] <= '2015-1-1')
  
# locate rows and access them using .loc() function
fifaRanks_filtered = fifaRanks.loc[fifaRanks_filtered]
  
# print dataframe
display(fifaRanks_filtered)

,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
995,43943,109,Estonia,EST,11,11,0,UEFA,1994-02-15
996,43916,165,St. Kitts and Nevis,SKN,0,0,1,CONCACAF,1994-02-15
997,44002,58,Slovakia,SVK,30,2,92,UEFA,1994-02-15
998,43817,57,IR Iran,IRN,30,31,2,AFC,1994-02-15
999,43826,165,Macau,MAC,0,0,1,AFC,1994-02-15
...,...,...,...,...,...,...,...,...,...
48965,44003,135,Tajikistan,TJK,215,215,0,AFC,2014-12-18
48966,43994,135,Azerbaijan,AZE,215,215,0,UEFA,2014-12-18
48967,43978,134,New Zealand,NZL,216,216,1,OFC,2014-12-18
48968,43862,133,Guinea-Bissau,GNB,218,218,2,CAF,2014-12-18


In [5]:
# Get year from rank_date and extract into new dataframe
fifaRanks_years = pd.DataFrame(fifaRanks_filtered['rank_date'].dt.year)

# Rename column name from rank_date to rank_year
fifaRanks_years.columns = ['rank_year']

display(fifaRanks_years)

# Get month from rank_date and extract into new dataframe
fifaRanks_months = pd.DataFrame(fifaRanks_filtered['rank_date'].dt.month)

# Rename column name from rank_date to rank_month
fifaRanks_months.columns = ['rank_month']

display(fifaRanks_months)

,rank_year
995,1994
996,1994
997,1994
998,1994
999,1994
...,...
48965,2014
48966,2014
48967,2014
48968,2014


,rank_month
995,2
996,2
997,2
998,2
999,2
...,...
48965,12
48966,12
48967,12
48968,12


In [6]:
#fifaRanks_filtered = fifaRanks_filtered.join(fifaRanks_months)

In [7]:
fifaRanks_filtered = fifaRanks_filtered.join(fifaRanks_years)

display(fifaRanks_filtered)

,id,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date,rank_year
995,43943,109,Estonia,EST,11,11,0,UEFA,1994-02-15,1994
996,43916,165,St. Kitts and Nevis,SKN,0,0,1,CONCACAF,1994-02-15,1994
997,44002,58,Slovakia,SVK,30,2,92,UEFA,1994-02-15,1994
998,43817,57,IR Iran,IRN,30,31,2,AFC,1994-02-15,1994
999,43826,165,Macau,MAC,0,0,1,AFC,1994-02-15,1994
...,...,...,...,...,...,...,...,...,...,...
48965,44003,135,Tajikistan,TJK,215,215,0,AFC,2014-12-18,2014
48966,43994,135,Azerbaijan,AZE,215,215,0,UEFA,2014-12-18,2014
48967,43978,134,New Zealand,NZL,216,216,1,OFC,2014-12-18,2014
48968,43862,133,Guinea-Bissau,GNB,218,218,2,CAF,2014-12-18,2014


In [8]:
#fifaRanks_filtered.groupby(['country_full', 'rank_date']).last()

#display(fifaRanks_filtered)

## 2 FIFA Groups and Matches

In [9]:
# View Matches dataset
fifa_all = pd.read_csv("WorldCupMatches.csv")

fifa_all.shape

display(fifa_all)

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning Matches Dataset

In [10]:
## Clean unnessary columns
fifa_clean = fifa_all.drop(["Datetime","Stadium", "City", "Attendance", "Referee", "Assistant 1", "Assistant 2", "MatchID"], axis=1)

## Remove trailing NAs
fifa_clean = fifa_clean.dropna(how='all')

# Change old team name of Cote d'Ivoire to Ivory Coast
fifa_clean[['Home Team Name', 'Away Team Name']] = fifa_clean[['Home Team Name', 'Away Team Name']].replace("C�te d'Ivoire", "Ivory Coast")

# Remove extra code chunk of " rn"> "
fifa_clean[['Home Team Name', 'Away Team Name']] = fifa_clean[['Home Team Name', 'Away Team Name']].replace({'rn">':''}, regex=True)

# Filter rows based on years from 1994 to 2014
newdf = (fifa_clean['Year'] >= 1994) & (fifa_clean['Year'] <= 2014)
  
# locate rows and access them using .loc() function
fifa_clean = fifa_clean.loc[newdf]

display(fifa_clean)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials
464,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR
465,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL
466,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI
467,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL
468,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU
...,...,...,...,...,...,...,...,...,...,...,...,...
847,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC
848,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER
849,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG
850,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED


***

## Merging Ranks and Matches Dataset

In [11]:
# Merge cleaned fifa matches dataset and fifaRanks dataset based on team names and year
fifaMatchesRanksHome = pd.merge(fifa_clean, fifaRanks_filtered,  how='left', left_on=['Home Team Name','Year'], right_on = ['country_full','rank_year'])

# Drop duplicate rank entries, keeping only the last entry for each year
fifaMatchesRanksHome.drop_duplicates(subset = ['Home Team Name', 'Away Team Name', 'RoundID', 'rank_year'], keep = 'last',  inplace= True) 


# Drop extra columns from fifaRanks dataset
fifaMatchesRanksHome = fifaMatchesRanksHome.drop(["id","country_full", "country_abrv", "confederation", "rank_date", "rank_year"], axis=1)
display(fifaMatchesRanksHome)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,rank,total_points,previous_points,rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC,5.0,1374.0,1374.0,0.0
4180,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER,6.0,1316.0,1316.0,0.0
4192,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG,5.0,1374.0,1374.0,0.0
4204,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED,6.0,1316.0,1316.0,0.0


In [12]:
fifaMatchesRanksHome = fifaMatchesRanksHome.rename(columns={col: 'home_'+col 
                        for col in fifaMatchesRanksHome.columns[range(12, 16, 1)]})
display(fifaMatchesRanksHome)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC,5.0,1374.0,1374.0,0.0
4180,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER,6.0,1316.0,1316.0,0.0
4192,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG,5.0,1374.0,1374.0,0.0
4204,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED,6.0,1316.0,1316.0,0.0


In [13]:
fifaMatchesRanksAway = pd.merge(fifaMatchesRanksHome, fifaRanks_filtered,  how='left', left_on=['Away Team Name','Year'], right_on = ['country_full','rank_year'])

# Drop duplicate rank entries, keeping only the last entry for each year
fifaMatchesRanksAway.drop_duplicates(subset = ['Home Team Name', 'Away Team Name', 'RoundID',  'rank_year'], keep = 'last',  inplace= True) 


# Drop extra columns from fifaRanks dataset
fifaMatchesRanksAway = fifaMatchesRanksAway.drop(["id","country_full", "country_abrv", "confederation", "rank_date", "rank_year"], axis=1)
display(fifaMatchesRanksAway)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,rank,total_points,previous_points,rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0,35.0,41.0,41.0,0.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0,44.0,37.0,37.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0,7.0,58.0,58.0,0.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0,9.0,58.0,58.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0,11.0,57.0,57.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC,5.0,1374.0,1374.0,0.0,16.0,997.0,995.0,0.0
3956,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER,6.0,1316.0,1316.0,0.0,1.0,1725.0,1725.0,0.0
3968,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG,5.0,1374.0,1374.0,0.0,2.0,1538.0,1538.0,0.0
3980,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED,6.0,1316.0,1316.0,0.0,5.0,1374.0,1374.0,0.0


In [14]:
fifaMatchesRanks = fifaMatchesRanksAway.rename(columns={col: 'away_'+col 
                        for col in fifaMatchesRanksAway.columns[range(16, 20, 1)]})
display(fifaMatchesRanks)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0,35.0,41.0,41.0,0.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0,44.0,37.0,37.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0,7.0,58.0,58.0,0.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0,9.0,58.0,58.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0,11.0,57.0,57.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC,5.0,1374.0,1374.0,0.0,16.0,997.0,995.0,0.0
3956,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER,6.0,1316.0,1316.0,0.0,1.0,1725.0,1725.0,0.0
3968,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG,5.0,1374.0,1374.0,0.0,2.0,1538.0,1538.0,0.0
3980,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED,6.0,1316.0,1316.0,0.0,5.0,1374.0,1374.0,0.0


In [15]:
# df1 =  fifa_clean.drop(fifa_clean.columns[[range(3, 12, 1)]], axis=1)

# display(df1)

# df2 =  fifaMatchesRanks.drop(fifaMatchesRanks.columns[[range(3, 17, 1)]], axis=1)

# display(df2)


# df_diff = pd.concat([df1,df2]).drop_duplicates(keep=False)

# df_diff

In [29]:
fifa_clean.to_csv('fifa_clean.csv', encoding='utf-8', index=False)

fifaMatchesRanks.to_csv('fifaMatchesRanks.csv', encoding='utf-8', index=False)


In [17]:
# df = fifaMatchesRanks.rename(columns={col: 'home_'+col 
#                         for col in fifaMatchesRanks.columns[range(12, 16, 1)]})
# display(df)

***

## Creating dataframes for Years 1994-2014

In [18]:
#def extract_fifayear(year):
    
    # only keep rows where Year == 2014
 #   fifayear = pd.DataFrame(fifaMatchesRanks.loc[fifaMatchesRanks['Year'] == year])
        
    # only keep Group "X" rows to get groups in FIFA 'year' Matches.
    #fifayear_groups = pd.DataFrame(fifayear.loc[:(fifayear == 'Round of 16').any(1).idxmax()])
        
    #drop duplicates
    #fifayear_groups_unique = pd.DataFrame(fifayear_groups.drop_duplicates(subset=['Home Team Name']))

    # alphabetical sort
    #fifayear_groups_unique = pd.DataFrame(fifayear_groups_unique.sort_values('Stage'))

    #remove all extra columns
    #fifayear_groups_unique = pd.DataFrame(
        #fifayear_groups_unique.drop(fifayear_groups_unique.columns[[range(3, 12, 1)]], axis=1))
         
    #return fifayear, fifayear_groups, fifayear_groups_unique

In [30]:
def extract_fifayear(year):
    
    # only keep rows where Year == 2014
    fifayear = pd.DataFrame(fifaMatchesRanks.loc[fifaMatchesRanks['Year'] == year])
        
    return fifayear

In [31]:
#fifa2014, fifa2014_groups, fifa2014_groups_unique = extract_fifayear(2014)

fifa2014 = extract_fifayear(2014)
display(fifa2014)

# display(fifa2014_groups)
# display(fifa2014_groups_unique)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
3236,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,BRA,CRO,6.0,1316.0,1316.0,0.0,19.0,946.0,946.0,0.0
3248,2014.0,Group A,Mexico,1.0,0.0,Cameroon,,0.0,0.0,255931.0,MEX,CMR,20.0,913.0,913.0,0.0,42.0,665.0,664.0,1.0
3260,2014.0,Group B,Spain,1.0,5.0,Netherlands,,1.0,1.0,255931.0,ESP,NED,9.0,1142.0,1142.0,0.0,5.0,1374.0,1374.0,0.0
3272,2014.0,Group B,Chile,3.0,1.0,Australia,,2.0,1.0,255931.0,CHI,AUS,14.0,1022.0,1022.0,0.0,100.0,324.0,327.0,2.0
3284,2014.0,Group C,Colombia,3.0,0.0,Greece,,1.0,0.0,255931.0,COL,GRE,3.0,1450.0,1450.0,0.0,24.0,856.0,856.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,NED,CRC,5.0,1374.0,1374.0,0.0,16.0,997.0,995.0,0.0
3956,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,BRA,GER,6.0,1316.0,1316.0,0.0,1.0,1725.0,1725.0,0.0
3968,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,NED,ARG,5.0,1374.0,1374.0,0.0,2.0,1538.0,1538.0,0.0
3980,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,BRA,NED,6.0,1316.0,1316.0,0.0,5.0,1374.0,1374.0,0.0


In [21]:
fifa2014.to_csv('fifa2014.csv', encoding='utf-8', index=False)

In [22]:
fifa2010 = extract_fifayear(2010)

display(fifa2010)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
2531,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,RSA,MEX,51.0,550.0,562.0,1.0,27.0,795.0,790.0,1.0
2542,2010.0,Group A,Uruguay,0.0,0.0,France,,0.0,0.0,249722.0,URU,FRA,7.0,1153.0,1174.0,0.0,18.0,867.0,852.0,3.0
2553,2010.0,Group B,Korea Republic,2.0,0.0,Greece,,1.0,0.0,249722.0,KOR,GRE,40.0,647.0,653.0,1.0,11.0,1016.0,1043.0,0.0
2564,2010.0,Group B,Argentina,1.0,0.0,Nigeria,,1.0,0.0,249722.0,ARG,NGA,5.0,1338.0,1353.0,0.0,32.0,725.0,711.0,1.0
2575,2010.0,Group C,England,1.0,1.0,USA,,1.0,1.0,249722.0,ENG,USA,6.0,1195.0,1250.0,0.0,18.0,867.0,847.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180,2010.0,Quarter-finals,Paraguay,0.0,1.0,Spain,,0.0,0.0,249718.0,PAR,ESP,24.0,832.0,850.0,1.0,1.0,1887.0,1920.0,0.0
3191,2010.0,Semi-finals,Uruguay,2.0,3.0,Netherlands,,1.0,1.0,249719.0,URU,NED,7.0,1153.0,1174.0,0.0,2.0,1723.0,1718.0,0.0
3202,2010.0,Semi-finals,Germany,0.0,1.0,Spain,,0.0,0.0,249719.0,GER,ESP,3.0,1485.0,1489.0,1.0,1.0,1887.0,1920.0,0.0
3213,2010.0,Match for third place,Uruguay,2.0,3.0,Germany,,1.0,1.0,249720.0,URU,GER,7.0,1153.0,1174.0,0.0,3.0,1485.0,1489.0,1.0


In [23]:
fifa2006 = extract_fifayear(2006)

display(fifa2006)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
1847,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,GER,CRC,6.0,1350.0,1348.0,0.0,68.0,458.0,458.0,0.0
1858,2006.0,Group A,Poland,0.0,2.0,Ecuador,,0.0,1.0,97410100.0,POL,ECU,24.0,854.0,859.0,2.0,30.0,802.0,802.0,0.0
1869,2006.0,Group B,England,1.0,0.0,Paraguay,,1.0,0.0,97410100.0,ENG,PAR,5.0,1359.0,1359.0,0.0,35.0,745.0,745.0,0.0
1880,2006.0,Group B,Trinidad and Tobago,0.0,0.0,Sweden,,0.0,0.0,97410100.0,TRI,SWE,91.0,360.0,355.0,1.0,14.0,958.0,958.0,0.0
1891,2006.0,Group C,Argentina,2.0,1.0,Ivory Coast,,2.0,0.0,97410100.0,ARG,CIV,3.0,1551.0,1551.0,0.0,18.0,919.0,919.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,2006.0,Quarter-finals,Brazil,0.0,1.0,France,,0.0,0.0,97410300.0,BRA,FRA,1.0,1588.0,1588.0,0.0,4.0,1523.0,1523.0,0.0
2487,2006.0,Semi-finals,Germany,0.0,2.0,Italy,Italy win after extra time,0.0,0.0,97410400.0,GER,ITA,6.0,1350.0,1348.0,0.0,2.0,1560.0,1560.0,0.0
2498,2006.0,Semi-finals,Portugal,0.0,1.0,France,,0.0,1.0,97410400.0,POR,FRA,8.0,1258.0,1258.0,0.0,4.0,1523.0,1523.0,0.0
2509,2006.0,Third place,Germany,3.0,1.0,Portugal,,0.0,0.0,97410500.0,GER,POR,6.0,1350.0,1348.0,0.0,8.0,1258.0,1258.0,0.0


In [32]:
fifa2002 = extract_fifayear(2002)

display(fifa2002)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
1143,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,FRA,SEN,2.0,787.0,787.0,0.0,27.0,646.0,640.0,2.0
1154,2002.0,Group A,Uruguay,1.0,2.0,Denmark,,0.0,1.0,43950100.0,URU,DEN,28.0,643.0,653.0,5.0,12.0,707.0,705.0,1.0
1165,2002.0,Group E,Republic of Ireland,1.0,1.0,Cameroon,,0.0,1.0,43950100.0,IRL,CMR,14.0,697.0,699.0,0.0,16.0,685.0,685.0,1.0
1176,2002.0,Group E,Germany,8.0,0.0,Saudi Arabia,,4.0,0.0,43950100.0,GER,KSA,4.0,761.0,763.0,0.0,38.0,608.0,612.0,0.0
1187,2002.0,Group F,Argentina,1.0,0.0,Nigeria,,0.0,0.0,43950100.0,ARG,NGA,5.0,751.0,741.0,0.0,29.0,642.0,643.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,2002.0,Quarter-finals,Spain,0.0,0.0,Korea Republic,Korea Republic win on penalties (3 - 5),0.0,0.0,43950300.0,ESP,KOR,3.0,779.0,779.0,0.0,20.0,669.0,671.0,0.0
1803,2002.0,Semi-finals,Germany,1.0,0.0,Korea Republic,,0.0,0.0,43950400.0,GER,KOR,4.0,761.0,763.0,0.0,20.0,669.0,671.0,0.0
1814,2002.0,Semi-finals,Brazil,1.0,0.0,Turkey,,0.0,0.0,43950400.0,BRA,TUR,1.0,856.0,857.0,0.0,9.0,729.0,728.0,1.0
1825,2002.0,Third place,Korea Republic,2.0,3.0,Turkey,,1.0,3.0,43950500.0,KOR,TUR,20.0,669.0,671.0,0.0,9.0,729.0,728.0,1.0


In [33]:
fifa1998 = extract_fifayear(1998)

display(fifa1998)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
529,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,BRA,SCO,1.0,73.0,74.0,0.0,38.0,50.0,50.0,2.0
539,1998.0,Group A,Morocco,2.0,2.0,Norway,,1.0,1.0,1014.0,MAR,NOR,13.0,60.0,60.0,0.0,14.0,59.0,59.0,0.0
549,1998.0,Group B,Italy,2.0,2.0,Chile,,1.0,1.0,1014.0,ITA,CHI,7.0,64.0,64.0,1.0,16.0,58.0,58.0,0.0
559,1998.0,Group B,Cameroon,1.0,1.0,Austria,,0.0,0.0,1014.0,CMR,AUT,41.0,49.0,48.0,2.0,22.0,55.0,55.0,1.0
569,1998.0,Group D,Paraguay,0.0,0.0,Bulgaria,,0.0,0.0,1014.0,PAR,BUL,25.0,55.0,55.0,1.0,49.0,45.0,46.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1998.0,Quarter-finals,Germany,0.0,3.0,Croatia,,0.0,1.0,1025.0,GER,CRO,3.0,67.0,67.0,0.0,4.0,67.0,67.0,0.0
1102,1998.0,Semi-finals,Brazil,1.0,1.0,Netherlands,Brazil win on penalties (4 - 2),0.0,0.0,1026.0,BRA,NED,1.0,73.0,74.0,0.0,11.0,62.0,62.0,2.0
1112,1998.0,Semi-finals,France,2.0,1.0,Croatia,,0.0,0.0,1026.0,FRA,CRO,2.0,69.0,69.0,0.0,4.0,67.0,67.0,0.0
1122,1998.0,Match for third place,Netherlands,1.0,2.0,Croatia,,1.0,2.0,1028.0,NED,CRO,11.0,62.0,62.0,2.0,4.0,67.0,67.0,0.0


In [34]:
fifa1994 = extract_fifayear(1994)

display(fifa1994)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0,35.0,41.0,41.0,0.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0,44.0,37.0,37.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0,7.0,58.0,58.0,0.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0,9.0,58.0,58.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0,11.0,57.0,57.0,0.0
59,1994.0,Group F,Belgium,1.0,0.0,Morocco,,1.0,0.0,337.0,BEL,MAR,24.0,47.0,48.0,2.0,33.0,43.0,43.0,2.0
69,1994.0,Group E,Norway,1.0,0.0,Mexico,,0.0,0.0,337.0,NOR,MEX,8.0,58.0,57.0,2.0,15.0,54.0,52.0,4.0
79,1994.0,Group B,Cameroon,2.0,2.0,Sweden,,1.0,1.0,337.0,CMR,SWE,31.0,44.0,44.0,3.0,3.0,61.0,62.0,1.0
89,1994.0,Group F,Netherlands,2.0,1.0,Saudi Arabia,,0.0,1.0,337.0,NED,KSA,6.0,60.0,60.0,0.0,27.0,45.0,45.0,0.0
99,1994.0,Group B,Brazil,2.0,0.0,Russia,,1.0,0.0,337.0,BRA,RUS,1.0,67.0,67.0,0.0,13.0,54.0,54.0,1.0


***

***

## World Cup Results Dataset

In [36]:
fifaResults = pd.read_csv("WorldCups.csv")

fifaResults.shape

display(fifaResults)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607
5,1958,Sweden,Brazil,Sweden,France,Germany FR,126,16,35,819.810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893.172
7,1966,England,England,Germany FR,Portugal,Soviet Union,89,16,32,1.563.135
8,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1.603.975
9,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1.865.753


In [37]:
fifaResults_filtered = (fifaResults['Year'] >= 1994)

# locate rows and access them using .loc() function
fifaResults_filtered = fifaResults.loc[fifaResults_filtered]

display(fifaResults_filtered)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
14,1994,USA,Brazil,Italy,Sweden,Bulgaria,141,24,52,3.587.538
15,1998,France,France,Brazil,Croatia,Netherlands,171,32,64,2.785.100
16,2002,Korea/Japan,Brazil,Germany,Turkey,Korea Republic,161,32,64,2.705.197
17,2006,Germany,Italy,France,Germany,Portugal,147,32,64,3.359.439
18,2010,South Africa,Spain,Netherlands,Germany,Uruguay,145,32,64,3.178.856
19,2014,Brazil,Germany,Argentina,Netherlands,Brazil,171,32,64,3.386.810


***

## Fairness Index Implementation

In [38]:
fifa1994 = extract_fifayear(1994)

display(fifa1994)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0,35.0,41.0,41.0,0.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0,44.0,37.0,37.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0,7.0,58.0,58.0,0.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0,9.0,58.0,58.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0,11.0,57.0,57.0,0.0
59,1994.0,Group F,Belgium,1.0,0.0,Morocco,,1.0,0.0,337.0,BEL,MAR,24.0,47.0,48.0,2.0,33.0,43.0,43.0,2.0
69,1994.0,Group E,Norway,1.0,0.0,Mexico,,0.0,0.0,337.0,NOR,MEX,8.0,58.0,57.0,2.0,15.0,54.0,52.0,4.0
79,1994.0,Group B,Cameroon,2.0,2.0,Sweden,,1.0,1.0,337.0,CMR,SWE,31.0,44.0,44.0,3.0,3.0,61.0,62.0,1.0
89,1994.0,Group F,Netherlands,2.0,1.0,Saudi Arabia,,0.0,1.0,337.0,NED,KSA,6.0,60.0,60.0,0.0,27.0,45.0,45.0,0.0
99,1994.0,Group B,Brazil,2.0,0.0,Russia,,1.0,0.0,337.0,BRA,RUS,1.0,67.0,67.0,0.0,13.0,54.0,54.0,1.0


## Equation

lm(result~points+fairness)

In [39]:
fifa1994.head()

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,Home Team Initials,Away Team Initials,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,ESP,KOR,2.0,62.0,61.0,3.0,35.0,41.0,41.0,0.0
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,GER,BOL,5.0,61.0,61.0,1.0,44.0,37.0,37.0,1.0
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,USA,SUI,23.0,48.0,48.0,1.0,7.0,58.0,58.0,0.0
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,ITA,IRL,4.0,61.0,61.0,1.0,9.0,58.0,58.0,1.0
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,COL,ROU,17.0,53.0,53.0,2.0,11.0,57.0,57.0,0.0


In [40]:
#### Create Fairness Score for Home and Away Teams

# Create a sklearn scaler object
mms = MinMaxScaler()
ss = StandardScaler()
norm = Normalizer(norm='l2')


def createFairnessIndex(fifayear):
    
    # For all values in the dataset, create fairness score based on home_total_points/away_total_points for the Home team
    # and vice-versa for the Away team
    
    #fifayear = fifayear.reset_index()
    
    for Stage, row in fifayear.iterrows():
        fifayear.loc[Stage,"Home Fairness Score"] = row['home_total_points'] / row['away_total_points']
        fifayear.loc[Stage,"Away Fairness Score"] = row['away_total_points'] / row['home_total_points']
        
        #fifayear = fifayear.dropna()

        #fifayear.replace([np.inf, -np.inf], np.nan, inplace=True)

        
        # Normalise data
        
        # Normalise the data to be between the range of 0-1. Lower scores closer to 0 indicate a fairer matchup, 
        # while higher scores closer to 1 indicate an unfair matchup. 

        #fifayear[['Home Fairness Score', 'Away Fairness Score']] = mms.fit_transform(
        #    fifayear[['Home Fairness Score', 'Away Fairness Score']], y = None)
        
        #fifayear[['Home Fairness Score', 'Away Fairness Score']] = scaler.fit_transform(
        #    fifayear[['Home Fairness Score', 'Away Fairness Score']])
        #fifayear = pd.DataFrame(norm.fit_transform(fifayear), columns=Away_Fairness_Score)
        
        
        
    return fifayear

In [41]:
def normaliseData(fifayear):
    
    fifayear[['Home Fairness Score', 'Away Fairness Score']] = mms.fit_transform(
            fifayear[['Home Fairness Score', 'Away Fairness Score']], y = None)  
    
    return fifayear

In [ ]:
# 
# Need total points for away teams.


#for Stage, row in fifa1994.iterrows():
#    fifa1994.loc[Stage,"Fairness Index"] = row['home_total_points'] / row['away_total_points']

#display(fifa1994)

In [42]:
fifa1994Fairness = createFairnessIndex(fifa1994)

#fifa1994Fairness = normaliseData(fifa1994)


display(fifa1994Fairness)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,...,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Fairness Score,Away Fairness Score
9,1994.0,Group C,Spain,2.0,2.0,Korea Republic,,0.0,0.0,337.0,...,2.0,62.0,61.0,3.0,35.0,41.0,41.0,0.0,1.512195,0.661290
19,1994.0,Group C,Germany,1.0,0.0,Bolivia,,0.0,0.0,337.0,...,5.0,61.0,61.0,1.0,44.0,37.0,37.0,1.0,1.648649,0.606557
29,1994.0,Group A,USA,1.0,1.0,Switzerland,,1.0,1.0,337.0,...,23.0,48.0,48.0,1.0,7.0,58.0,58.0,0.0,0.827586,1.208333
39,1994.0,Group E,Italy,0.0,1.0,Republic of Ireland,,0.0,1.0,337.0,...,4.0,61.0,61.0,1.0,9.0,58.0,58.0,1.0,1.051724,0.950820
49,1994.0,Group A,Colombia,1.0,3.0,Romania,,1.0,2.0,337.0,...,17.0,53.0,53.0,2.0,11.0,57.0,57.0,0.0,0.929825,1.075472
59,1994.0,Group F,Belgium,1.0,0.0,Morocco,,1.0,0.0,337.0,...,24.0,47.0,48.0,2.0,33.0,43.0,43.0,2.0,1.093023,0.914894
69,1994.0,Group E,Norway,1.0,0.0,Mexico,,0.0,0.0,337.0,...,8.0,58.0,57.0,2.0,15.0,54.0,52.0,4.0,1.074074,0.931034
79,1994.0,Group B,Cameroon,2.0,2.0,Sweden,,1.0,1.0,337.0,...,31.0,44.0,44.0,3.0,3.0,61.0,62.0,1.0,0.721311,1.386364
89,1994.0,Group F,Netherlands,2.0,1.0,Saudi Arabia,,0.0,1.0,337.0,...,6.0,60.0,60.0,0.0,27.0,45.0,45.0,0.0,1.333333,0.750000
99,1994.0,Group B,Brazil,2.0,0.0,Russia,,1.0,0.0,337.0,...,1.0,67.0,67.0,0.0,13.0,54.0,54.0,1.0,1.240741,0.805970


In [45]:
#fifa1994Fairness.to_csv('fifa1994Fairness.csv', encoding='utf-8', index=False)

#fifa1998Fairness.to_csv('fifa1998Fairness.csv', encoding='utf-8', index=False)

#print(fifa2014Fairness.info()) # will give you info about the dataset columns

Average out fairness score for each team in the GROUP STAGE every year

fairness for each home team and away teams, based on opposite team points

combine scores so hoome and away difference does not exist

In [46]:
fifa1998Fairness = createFairnessIndex(fifa1998)

#fifa1998Fairness = normaliseData(fifa1998)


display(fifa1998Fairness)

#fifa1998Fairness.to_csv('fifa1998Fairness.csv', encoding='utf-8', index=False)


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,...,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Fairness Score,Away Fairness Score
529,1998.0,Group A,Brazil,2.0,1.0,Scotland,,1.0,1.0,1014.0,...,1.0,73.0,74.0,0.0,38.0,50.0,50.0,2.0,1.460000,0.684932
539,1998.0,Group A,Morocco,2.0,2.0,Norway,,1.0,1.0,1014.0,...,13.0,60.0,60.0,0.0,14.0,59.0,59.0,0.0,1.016949,0.983333
549,1998.0,Group B,Italy,2.0,2.0,Chile,,1.0,1.0,1014.0,...,7.0,64.0,64.0,1.0,16.0,58.0,58.0,0.0,1.103448,0.906250
559,1998.0,Group B,Cameroon,1.0,1.0,Austria,,0.0,0.0,1014.0,...,41.0,49.0,48.0,2.0,22.0,55.0,55.0,1.0,0.890909,1.122449
569,1998.0,Group D,Paraguay,0.0,0.0,Bulgaria,,0.0,0.0,1014.0,...,25.0,55.0,55.0,1.0,49.0,45.0,46.0,3.0,1.222222,0.818182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1998.0,Quarter-finals,Germany,0.0,3.0,Croatia,,0.0,1.0,1025.0,...,3.0,67.0,67.0,0.0,4.0,67.0,67.0,0.0,1.000000,1.000000
1102,1998.0,Semi-finals,Brazil,1.0,1.0,Netherlands,Brazil win on penalties (4 - 2),0.0,0.0,1026.0,...,1.0,73.0,74.0,0.0,11.0,62.0,62.0,2.0,1.177419,0.849315
1112,1998.0,Semi-finals,France,2.0,1.0,Croatia,,0.0,0.0,1026.0,...,2.0,69.0,69.0,0.0,4.0,67.0,67.0,0.0,1.029851,0.971014
1122,1998.0,Match for third place,Netherlands,1.0,2.0,Croatia,,1.0,2.0,1028.0,...,11.0,62.0,62.0,2.0,4.0,67.0,67.0,0.0,0.925373,1.080645


In [47]:
fifa2002Fairness = createFairnessIndex(fifa2002)

display(fifa2002Fairness)

fifa2002Fairness.to_csv('fifa2002Fairness.csv', encoding='utf-8', index=False)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,...,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Fairness Score,Away Fairness Score
1143,2002.0,Group A,France,0.0,1.0,Senegal,,0.0,1.0,43950100.0,...,2.0,787.0,787.0,0.0,27.0,646.0,640.0,2.0,1.218266,0.820839
1154,2002.0,Group A,Uruguay,1.0,2.0,Denmark,,0.0,1.0,43950100.0,...,28.0,643.0,653.0,5.0,12.0,707.0,705.0,1.0,0.909477,1.099533
1165,2002.0,Group E,Republic of Ireland,1.0,1.0,Cameroon,,0.0,1.0,43950100.0,...,14.0,697.0,699.0,0.0,16.0,685.0,685.0,1.0,1.017518,0.982783
1176,2002.0,Group E,Germany,8.0,0.0,Saudi Arabia,,4.0,0.0,43950100.0,...,4.0,761.0,763.0,0.0,38.0,608.0,612.0,0.0,1.251645,0.798949
1187,2002.0,Group F,Argentina,1.0,0.0,Nigeria,,0.0,0.0,43950100.0,...,5.0,751.0,741.0,0.0,29.0,642.0,643.0,2.0,1.169782,0.854860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,2002.0,Quarter-finals,Spain,0.0,0.0,Korea Republic,Korea Republic win on penalties (3 - 5),0.0,0.0,43950300.0,...,3.0,779.0,779.0,0.0,20.0,669.0,671.0,0.0,1.164425,0.858793
1803,2002.0,Semi-finals,Germany,1.0,0.0,Korea Republic,,0.0,0.0,43950400.0,...,4.0,761.0,763.0,0.0,20.0,669.0,671.0,0.0,1.137519,0.879106
1814,2002.0,Semi-finals,Brazil,1.0,0.0,Turkey,,0.0,0.0,43950400.0,...,1.0,856.0,857.0,0.0,9.0,729.0,728.0,1.0,1.174211,0.851636
1825,2002.0,Third place,Korea Republic,2.0,3.0,Turkey,,1.0,3.0,43950500.0,...,20.0,669.0,671.0,0.0,9.0,729.0,728.0,1.0,0.917695,1.089686


In [48]:
fifa2006Fairness = createFairnessIndex(fifa2006)

display(fifa2006Fairness)

fifa2006Fairness.to_csv('fifa2006Fairness.csv', encoding='utf-8', index=False)


,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,...,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Fairness Score,Away Fairness Score
1847,2006.0,Group A,Germany,4.0,2.0,Costa Rica,,2.0,1.0,97410100.0,...,6.0,1350.0,1348.0,0.0,68.0,458.0,458.0,0.0,2.947598,0.339259
1858,2006.0,Group A,Poland,0.0,2.0,Ecuador,,0.0,1.0,97410100.0,...,24.0,854.0,859.0,2.0,30.0,802.0,802.0,0.0,1.064838,0.939110
1869,2006.0,Group B,England,1.0,0.0,Paraguay,,1.0,0.0,97410100.0,...,5.0,1359.0,1359.0,0.0,35.0,745.0,745.0,0.0,1.824161,0.548197
1880,2006.0,Group B,Trinidad and Tobago,0.0,0.0,Sweden,,0.0,0.0,97410100.0,...,91.0,360.0,355.0,1.0,14.0,958.0,958.0,0.0,0.375783,2.661111
1891,2006.0,Group C,Argentina,2.0,1.0,Ivory Coast,,2.0,0.0,97410100.0,...,3.0,1551.0,1551.0,0.0,18.0,919.0,919.0,0.0,1.687704,0.592521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476,2006.0,Quarter-finals,Brazil,0.0,1.0,France,,0.0,0.0,97410300.0,...,1.0,1588.0,1588.0,0.0,4.0,1523.0,1523.0,0.0,1.042679,0.959068
2487,2006.0,Semi-finals,Germany,0.0,2.0,Italy,Italy win after extra time,0.0,0.0,97410400.0,...,6.0,1350.0,1348.0,0.0,2.0,1560.0,1560.0,0.0,0.865385,1.155556
2498,2006.0,Semi-finals,Portugal,0.0,1.0,France,,0.0,1.0,97410400.0,...,8.0,1258.0,1258.0,0.0,4.0,1523.0,1523.0,0.0,0.826001,1.210652
2509,2006.0,Third place,Germany,3.0,1.0,Portugal,,0.0,0.0,97410500.0,...,6.0,1350.0,1348.0,0.0,8.0,1258.0,1258.0,0.0,1.073132,0.931852


In [49]:
fifa2010Fairness = createFairnessIndex(fifa2010)

display(fifa2010Fairness)

fifa2010Fairness.to_csv('fifa2010Fairness.csv', encoding='utf-8', index=False)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,...,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Fairness Score,Away Fairness Score
2531,2010.0,Group A,South Africa,1.0,1.0,Mexico,,0.0,0.0,249722.0,...,51.0,550.0,562.0,1.0,27.0,795.0,790.0,1.0,0.691824,1.445455
2542,2010.0,Group A,Uruguay,0.0,0.0,France,,0.0,0.0,249722.0,...,7.0,1153.0,1174.0,0.0,18.0,867.0,852.0,3.0,1.329873,0.751951
2553,2010.0,Group B,Korea Republic,2.0,0.0,Greece,,1.0,0.0,249722.0,...,40.0,647.0,653.0,1.0,11.0,1016.0,1043.0,0.0,0.636811,1.570325
2564,2010.0,Group B,Argentina,1.0,0.0,Nigeria,,1.0,0.0,249722.0,...,5.0,1338.0,1353.0,0.0,32.0,725.0,711.0,1.0,1.845517,0.541854
2575,2010.0,Group C,England,1.0,1.0,USA,,1.0,1.0,249722.0,...,6.0,1195.0,1250.0,0.0,18.0,867.0,847.0,6.0,1.378316,0.725523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3180,2010.0,Quarter-finals,Paraguay,0.0,1.0,Spain,,0.0,0.0,249718.0,...,24.0,832.0,850.0,1.0,1.0,1887.0,1920.0,0.0,0.440911,2.268029
3191,2010.0,Semi-finals,Uruguay,2.0,3.0,Netherlands,,1.0,1.0,249719.0,...,7.0,1153.0,1174.0,0.0,2.0,1723.0,1718.0,0.0,0.669182,1.494363
3202,2010.0,Semi-finals,Germany,0.0,1.0,Spain,,0.0,0.0,249719.0,...,3.0,1485.0,1489.0,1.0,1.0,1887.0,1920.0,0.0,0.786963,1.270707
3213,2010.0,Match for third place,Uruguay,2.0,3.0,Germany,,1.0,1.0,249720.0,...,7.0,1153.0,1174.0,0.0,3.0,1485.0,1489.0,1.0,0.776431,1.287944


In [50]:
fifa2014Fairness = createFairnessIndex(fifa2014)

display(fifa2014Fairness)

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,RoundID,...,home_rank,home_total_points,home_previous_points,home_rank_change,away_rank,away_total_points,away_previous_points,away_rank_change,Home Fairness Score,Away Fairness Score
3236,2014.0,Group A,Brazil,3.0,1.0,Croatia,,1.0,1.0,255931.0,...,6.0,1316.0,1316.0,0.0,19.0,946.0,946.0,0.0,1.391121,0.718845
3248,2014.0,Group A,Mexico,1.0,0.0,Cameroon,,0.0,0.0,255931.0,...,20.0,913.0,913.0,0.0,42.0,665.0,664.0,1.0,1.372932,0.728368
3260,2014.0,Group B,Spain,1.0,5.0,Netherlands,,1.0,1.0,255931.0,...,9.0,1142.0,1142.0,0.0,5.0,1374.0,1374.0,0.0,0.831150,1.203152
3272,2014.0,Group B,Chile,3.0,1.0,Australia,,2.0,1.0,255931.0,...,14.0,1022.0,1022.0,0.0,100.0,324.0,327.0,2.0,3.154321,0.317025
3284,2014.0,Group C,Colombia,3.0,0.0,Greece,,1.0,0.0,255931.0,...,3.0,1450.0,1450.0,0.0,24.0,856.0,856.0,1.0,1.693925,0.590345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,2014.0,Quarter-finals,Netherlands,0.0,0.0,Costa Rica,Netherlands win on penalties (4 - 3),0.0,0.0,255953.0,...,5.0,1374.0,1374.0,0.0,16.0,997.0,995.0,0.0,1.378134,0.725619
3956,2014.0,Semi-finals,Brazil,1.0,7.0,Germany,,0.0,5.0,255955.0,...,6.0,1316.0,1316.0,0.0,1.0,1725.0,1725.0,0.0,0.762899,1.310790
3968,2014.0,Semi-finals,Netherlands,0.0,0.0,Argentina,Argentina win on penalties (2 - 4),0.0,0.0,255955.0,...,5.0,1374.0,1374.0,0.0,2.0,1538.0,1538.0,0.0,0.893368,1.119360
3980,2014.0,Play-off for third place,Brazil,0.0,3.0,Netherlands,,0.0,2.0,255957.0,...,6.0,1316.0,1316.0,0.0,5.0,1374.0,1374.0,0.0,0.957787,1.044073


In [51]:
fifa2014Fairness.to_csv('fifa2014Fairness.csv', encoding='utf-8', index=False)


        =Mean(Points[Group==C2&!Team==A2])
***

In [ ]:
#def generateFairness(fifayear):
        # only keep Group "X" rows to get groups in FIFA 'year' Matches.
#        fifayear_groups = fifayear[fifayear['Stage'].str.contains('Group', regex=False)]#

 #       fifaTeams = pd.DataFrame(fifayear_groups['Home Team Name'].unique())
 #       fifaTeams.columns = ['Team Name']

 #       def calculateFairness(fifaTeams):
#            fairness_Sum = fifayear_groups["Home Team Name"]==fifaTeams
 #           return()
        
        
 #       return fifayear_groups
        

In [ ]:
#fifa2014FairnessNew = generateFairness(fifa2014Fairness)
#display(fifa2014FairnessNew)

In [ ]:
#uniqueValues = pd.DataFrame(fifa2014FairnessNew['Home Team Name'].unique())
#uniqueValues.columns = ['Team Name']

#uniqueValues.head()

In [ ]:
#uniqueValuesAway = pd.DataFrame(fifa2014FairnessNew[fifa2014FairnessNew['away_total_points'].unique()])
#uniqueValuesAway

#uniqueValues.head()

In [52]:
import rpy2.robjects as robjects
robjects.r.source("FairnessCalc.R")

value,[RTYPES.VECSXP]
visible,[RTYPES.LGLSXP]


In [53]:
def generateFairness(year):
    
    # Convert Year input into 'string' type
    year = str(year)
    
    # Extract matching fairness dataframe from the R script into a dataframe
    fifayearFairness = pd.DataFrame(robjects.globalenv["fifa"+year+"Fairness"]).T

    # Rename column names
    fifayearFairness.columns = ['Country', 'Fairness']

    return(fifayearFairness)

In [54]:
fifa1994FairnessScore = generateFairness(1994)
fifa1998FairnessScore = generateFairness(1998)
fifa2002FairnessScore = generateFairness(2002)
fifa2006FairnessScore = generateFairness(2006)
fifa2010FairnessScore = generateFairness(2010)
fifa2014FairnessScore = generateFairness(2014)

In [55]:
fifa2014FairnessScore.head(10)

,Country,Fairness
0,Algeria,2687.0
1,Argentina,2053.0
2,Australia,3538.0
3,Belgium,2218.0
4,Bosnia and Herzegovina,2774.0
5,Brazil,2524.0
6,Cameroon,3175.0
7,Chile,2840.0
8,Colombia,2252.0
9,Costa Rica,3270.0


In [60]:
fifa1998FairnessScore.head(10)

,Country,Fairness
0,Argentina,174.0
1,Belgium,181.0
2,Brazil,169.0
3,Cameroon,177.0
4,Chile,168.0
5,Colombia,178.0
6,England,167.0
7,France,162.0
8,Germany,NaN
9,Italy,162.0


In [63]:
# View Matches dataset
fifa_2018 = pd.read_csv("World_cup_2018_matches.csv")

fifa_2018.shape

display(fifa_2018)

,Date,Stage,Home,Away,Home_goals,Away_goals,Home_shots,Home_shots_on_target,Home_possession,Home_fouls,...,Away_offsides,Away_corners,Extra_time,Home_extra_goals,Away_extra_goals,Penalties,Home_PK_taken,Home_PK_made,Away_PK_taken,Away_PK_made
0,2018-06-14,Group-1,Russia,Saudi Arabia,5,0,14,7,38,22,...,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-15,Group-1,Egypt,Uruguay,0,1,8,3,41,12,...,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-15,Group-1,Morocco,Iran,0,1,13,3,68,22,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-15,Group-1,Portugal,Spain,3,3,9,3,33,12,...,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-06-16,Group-1,France,Australia,2,1,12,5,55,16,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2018-07-07,QFinals,Russia,Croatia,2,2,13,5,35,25,...,0,8,Y,1.0,1.0,Y,5.0,3.0,5.0,4.0
60,2018-07-10,SFinals,France,Belgium,1,0,19,5,36,6,...,1,5,N,NaN,NaN,N,NaN,NaN,NaN,NaN
61,2018-07-11,SFinals,Croatia,England,2,1,22,7,56,23,...,3,4,Y,1.0,0.0,N,NaN,NaN,NaN,NaN
62,2018-07-14,Third-Place,Belgium,England,2,0,12,4,42,11,...,0,5,N,NaN,NaN,N,NaN,NaN,NaN,NaN


In [ ]:
##
## Calculate the fairness index for 2018 Data Set 

In [65]:
#1. Calculating 2018 data set fairness index.
#2.  Next add all fairness Index into a new dataset.
#3. Next add the final team rankings into the dataset.
#4. Next Perform Visualisations and Insights.
#5. Next Perform Modelling and Multiple linear regression.
#6. Next Perform Simulation to predict the winner for draw.
#7. Next Predict who wins the 2022 FIFA world cup.

In [66]:
fifa2018Fairness = createFairnessIndex(fifa_2018)


KeyError: 'home_total_points'